In [1]:
# Azure storage package 설치
%pip install azure-storage-blob azure-identity

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# Blob Serveices에 접속

connect_str = 'DefaultEndpointsProtocol=https;AccountName=labuser10storage1;AccountKey=yV7rRbPg7ykSrPqTvVJc8IOOJcMnTVscpV1D8tOG0HpJ1xJI4bA6fxz/6SHFJpuP1J6V+xfX2r6A+AStA3qaJQ==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
# BlobServiceClient 에게 명령을 줌
# 이름 하고 key가 필요함
# blob_service_client가 BlobServiceClient에 연결된 상태를 나타내는 객체가 됨

In [4]:
# Container 만들기

container_name = 'test2container'

container_client = blob_service_client.create_container(container_name)
# 소문자로 다 통일해서 쓰는게 나중에 오류를 막을 수 있다.
# container_client에 값을 받아서 쓸 수 있도록 만듬

# container가 azure container에 생성된 걸 확인 할 수 있음!

In [12]:
# local에서 text 파일 업로드 하기

# Create a local directory to hold blob data
local_path = '../99_Src/Azure/data'
os.mkdir(local_path)  ## jupyter 계정의 폴더에 디렉토리 만들어줌
## os make directory
## os module이 os 역활을 수행해줌 : 디렉토리 만들고 수정하는 작업.

# Create a file in the local data directory to upload and download
local_file_name = 'welcome'+'.txt' ## 파일이름을 welcome, txt 파일로 생성
upload_file_path = os.path.join(local_path, local_file_name) # join 디렉토리와 파일을 연결해줌

# Write text to the file
file = open(upload_file_path, mode='w') ## 파일을 열어줌, 없으면 만들어서 열어줌
file.write('welcome to python')
## colab 파일 폴더에 welcome.txt가 생긴 걸 확인 할 수 있다.
file.close() ## 모든 작업을 다 끝내고 꼭 close 해야함


In [13]:
blob_client = blob_service_client.get_blob_client(
                  container=container_name, blob=local_file_name)

with open(file=upload_file_path, mode='rb') as data:
    # file을 열 때 쓰는 기능
    # rb가 중요함.
  blob_client.upload_blob(data)

txt 파일 : 텍스트가 들어있는 파일 -> 사용자 친화
binary 파일 : 실행파일과 바이너리 형식의 텍스트 파일 -> 컴퓨터 친화
 
python은 txt 파일 -> open -> r(read) -> b(as binary file) -> as data로 저장
upload_blob을 통해서 blob container에 업로드
만약 wb는 write as binary.

In [18]:
# blob 파일의 리스트를 확인한다.

print('\nListing blobs')

blob_list = container_client.list_blobs()
blob_list # 파일의 목록이 있는 목록 객체

for blob in blob_list:  # for문을 이용해서 blob목록을 출력
    print('\t' + blob.name)


Listing blobs
	welcome.txt


In [22]:
# Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extension 

download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name, '.txt','_DOWNLOAD.txt')) # _download.txt로 바뀌는 method / replace(a,'b','c') -> a.b 를 a.c로 치환
print(download_file_path) # 다운로드 받을 파일의 이름을 결정함
container_client = blob_service_client.get_container_client(container=container_name)

with open(file=download_file_path, mode='wb') as download_file:  # 다운로드를 받을  수 있는 method + 다운받은 파일의 이름을 지정
    download_file.write(container_client.download_blob(blob.name).readall())   # blob container에서 파일을 통채로 다운받고 읽어 들이는 과정.
    
    # welcome.txt를 container에 업로드하고, 그 파일이름을 download.txt로 바꿔서 다운로드 하는 작업

../99_Src/Azure/data\welcome_DOWNLOAD.txt


In [23]:
# Clean Up

print('\n Press the Enter Key to begin clean up')
input()

# if input() == 'yes':
# if문을 사용하면 특정단어를 사용해야만 삭제할 수 있다.

print('Deleting blob container...')
container_client.delete_container() # Container 삭제하는 작업

print('Deleting the local source and download files...')    # 파일을 지우고 디렉토리를 삭제하는 작업
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)

print('Done')

# 작업이 끝나고 Resource 정리할 때 사용


 Press the Enter Key to begin clean up
Deleting blob container...
Deleting the local source and download files...
Done


데이터는 가상머신의 디스크에 저장하면 안됨..
-> 언제든지 초기화 될 수 있음.

그래서 storage 계정을 따로 만들어야함..!

가상 서버를 만들 때는 두 대를  묶어서 사용해야함 -> 가용성 집합
한 대는 관리 한 대는 서비스 지속